<h1 align="center"> ADA - Milestone 2</h1> 

**Students:**
- ABI FADEL Zad
- ADEYE Abiola
- BRUNO Etienne
- FERCHIOU Sami

In [ ]:
# Import libraries
import sqlite3

import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import numpy as np
import pandas as pd
import seaborn as sns
from dask.distributed import Client
from IPython.display import HTML, display
from matplotlib import pyplot as plt

display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
nb_workers = multiprocessing.cpu_count()
client = Client(n_workers=nb_workers)
client

# How did popularity of basketball evolve over time ?

In [ ]:
video_metadata_df = dd.read_parquet("./data/video_metadata/parquet")
video_metadata_df["upload_date"] = dd.to_datetime(video_metadata_df.upload_date)

In [ ]:
video_metadata_df.head()

In [ ]:
date_obj = video_metadata_df.upload_date.dt
group_by = [date_obj.year, date_obj.month, video_metadata_df.categories]

video_evolution = video_metadata_df[["upload_date"]].groupby(by=group_by).count().persist()
video_evolution_pd = video_evolution.compute()

video_evolution_pd.sort_index(inplace=True)
# video_evolution_pd = video_evolution_pd[video_evolution.index.dt.year != 2019]

video_evolution_pd.index.names = ["year", "month", "categories"]
video_evolution_pd.reset_index(inplace=True)
video_evolution_pd = video_evolution_pd[(video_evolution_pd["year"] != 2019)]  # TODO : fix this to also include first months of 2019
video_evolution_pd["date"] = video_evolution_pd["year"].astype("str") + "/" + video_evolution_pd["month"].astype("str")

In [ ]:
def plot_comparsion(df):
    fig = plt.figure(figsize=(15, 8))
    ax = fig.add_subplot(111)
    sns.lineplot(x="date", y="upload_date", hue="categories", data=df)
    start, end = ax.get_xlim()
    ax.xaxis.set_ticks(np.arange(start, end, 12))
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
video_evolution_pd_sport = video_evolution_pd[video_evolution_pd["categories"].isin(["Sports", "People & Blogs", "Entertainment"])]

In [ ]:
plot_comparsion(video_evolution_pd)

In [ ]:
plot_comparsion(video_evolution_pd_sport)

In [ ]:
video_metadata_df.count().compute()

In [ ]:
video_metadata_df_sports = video_metadata_df[video_metadata_df["categories"].isin(["Sports", "People & Blogs", "Entertainment"])]
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].str.lower()
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].apply(lambda x: f",{x},")

In [ ]:
# video_metadata_df_sports = video_metadata_df_sports[video_metadata_df_sports['tags'].str.contains('basketball')]


date_obj = video_metadata_df_sports.upload_date.dt
group_by = [date_obj.year, date_obj.month, video_metadata_df.categories]

video_metadata_df_sports_grouped = video_metadata_df_sports[["upload_date"]].groupby(by=group_by).count().persist()

In [ ]:
video_metadata_df_sports.head()

# Big Market vs Small Market Teams Case Study

In [ ]:
video_metadata_df_sports.head()

In [ ]:
required_tags = ["nba", "basketball"]
nba_basketball_df = video_metadata_df_sports[video_metadata_df_sports["tags"].str.contains(f',{",|,".join(required_tags)},')].persist()
nba_basketball_df["year_month"] = nba_basketball_df.upload_date.dt.year.astype("str") + "-" + nba_basketball_df.upload_date.dt.month.astype("str")

In [ ]:
nba_basketball_df.head()

In [ ]:
BigMarkets = {
    "LA": ["los angeles", "la", "lakers"],
    "NY": ["knicks", "new york", "ny"],
    "CH": ["bulls", "chicago"],
    "GSW": [
        "gsw",
        "golden state warriors",
        "golden state",
        "warriors",
        "dubs",
        "dubs nation",
        "dubsnation",
    ],
}

SmallMarkets = {
    "ME": ["memphis", "grizzlies"],
    "NO": ["pelicans", "pels", "neworleans", "new orleans", "nola"],
    "OKC": ["okc", "oklahomacitythunder", "oklahoma"],
    "MB": ["bucks", "milwaukee"],
}


Correspondence = {
    "Chicago Bulls": "CH",
    "Los Angeles Lakers": "LA",
    "Memphis Grizzlies": "ME",
    "Milwaukee Bucks": "MB",
    "New Orleans Pelicans": "NO",
    "New York Knicks": "NY",
    "Oklahoma City Thunder": "OKC",
    "Golden State Warriors": "GSW",
}

nba_basketball_views_per_year_df = nba_basketball_df[["year_month", "view_count"]].groupby("year_month").sum().persist().compute()
nba_basketball_views_per_year_df.rename(columns={"view_count": "total_nba_views"}, inplace=True)

dd_dict = {}

# Getting the total amount of views per month for every big market and small market team
for key, value in {**BigMarkets, **SmallMarkets}.items():

    team_df = nba_basketball_df[nba_basketball_df["tags"].str.contains(f',{",|,".join(value)},')]
    # team_df["year_month"] = team_df.upload_date.dt.year.astype("str") + "-" + team_df.upload_date.dt.month.astype("str")
    team_df = team_df[["year_month", "view_count"]].groupby("year_month").sum()

    team_df = team_df.reset_index()

    team_df["team"] = key
    team_df["market_size"] = "small" if key in SmallMarkets else "big"
    team_df = team_df.persist().compute()

    dd_dict[key] = team_df

In [ ]:
all_team_results = pd.concat([v for k, v in dd_dict.items()])
all_team_results = all_team_results.merge(nba_basketball_views_per_year_df, on="year_month")
all_team_results["total_nba_view_percentage"] = all_team_results["view_count"] / all_team_results["total_nba_views"]

In [ ]:
all_team_results.head()

In [ ]:
con = sqlite3.connect("data/nba_api.sqlite")

In [ ]:
occupancy_rate_df = pd.read_sql_query(
    """select Home as home, avg(Attendance) as avg_attendace, round(avg(Attendance/Capacity*100),2) as avg_occupancy_rate, strftime('%Y-%m',Time) as year_month
from game_data
group by Home, year_month""",
    con,
)
occupancy_rate_df = occupancy_rate_df.reset_index()

occupancy_rate_df = occupancy_rate_df[occupancy_rate_df["home"].isin(Correspondence.keys())]


winning_rate_df = pd.read_sql_query(
    """
with games as (select *, Visitor as team
               from game_data
               union all
               select *, Home as team
               from game_data)
select team,
       avg(case when "V PTS" > "H PTS" then 1 else 0 end) as win_percentage,
       strftime('%Y-%m', Time)                            as year_month
from games
group by team, year_month
""",
    con,
)
winning_rate_df = winning_rate_df.reset_index()
winning_rate_df = winning_rate_df[winning_rate_df["team"].isin(Correspondence.keys())]

In [ ]:
winning_rate_df["team"] = winning_rate_df["team"].apply(lambda x: Correspondence[x])
occupancy_rate_df["team"] = occupancy_rate_df["home"].apply(lambda x: Correspondence[x])

In [ ]:
winning_rate_df.head()

In [ ]:
occupancy_rate_df.head()

In [ ]:
all_team_results = all_team_results.merge(winning_rate_df.drop(["index"], axis=1), on=["team", "year_month"])
all_team_results = all_team_results.merge(occupancy_rate_df.drop(["index", "home"], axis=1), on=["team", "year_month"])

In [ ]:
all_team_results.head()